**基本介绍**

FastText是Facebook公司2016年开源的项目，主要用于构建词向量和文本分类。FastText最大的优点是又快又好，因为本身结构非常简单，只有一层浅层网络，所以模型训练和线上预测的速度非常快，在普通的CPU上也能达到分钟级别的训练速度。不仅如此，在很多分类任务的标准数据集上模型效果也很不错。正因为速度非常快，并且模型效果还不错，所以在工业界应用范围很广。

FastText主要用于**训练词向量和文本分类**任务。

论文：Bag of Tricks for Efficient Text Classification

**1、fastText与CBoW之间的关系?**



fastText在训练词向量时，中为了将顺序信息，同时使用了word n-gram，即subWords作为输入，但由于n-gram过多，又采用buckets，所以fastText，输入矩阵的大小为 (N-Words + N-Buckets) * dim。

训练词向量时，最终词向量的表示包括：词+word n-gram。



In [ ]:
# 参考 https://heleifz.github.io/14732610572844.html

void FastText::cbow(Model& model, real lr,
                    const std::vector<int32_t>& line) {
  std::vector<int32_t> bow;
  std::uniform_int_distribution<> uniform(1, args_->ws);
  
  // 在一个句子中，每个词可以进行一次 update
  for (int32_t w = 0; w < line.size(); w++) {
    // 一个词的上下文长度是随机产生的
    int32_t boundary = uniform(model.rng);
    bow.clear();
    // 以当前词为中心，将左右 boundary 个词加入 input
    for (int32_t c = -boundary; c <= boundary; c++) {
      // 当然，不能数组越界
      if (c != 0 && w + c >= 0 && w + c < line.size()) {
        // 实际被加入 input 的不止是词本身，还有词的 word n-gram
        const std::vector<int32_t>& ngrams = dict_->getNgrams(line[w + c]);
        bow.insert(bow.end(), ngrams.cbegin(), ngrams.cend());
      }
    }
    // 完成一次 CBOW 更新
    model.update(bow, line[w], lr);
  }
}

void FastText::skipgram(Model& model, real lr,
                        const std::vector<int32_t>& line) {
  std::uniform_int_distribution<> uniform(1, args_->ws);
  for (int32_t w = 0; w < line.size(); w++) {
    // 一个词的上下文长度是随机产生的
    int32_t boundary = uniform(model.rng);
    // 采用 词+word n-gram 来预测这个词的上下文的所有的词
    const std::vector<int32_t>& ngrams = dict_->getNgrams(line[w]);
    // 在 skipgram 中，对上下文的每一个词分别更新一次模型
    for (int32_t c = -boundary; c <= boundary; c++) {
      if (c != 0 && w + c >= 0 && w + c < line.size()) {
        model.update(ngrams, line[w + c], lr);
      }
    }
  }
}


# 从上面skip-gram和cbow的c代码就能看出，skip-gram是每个context词都更新，故而对低频词更加敏感，训练效果更好。小数据集更容易过拟合。

In [5]:
import fasttext
import pandas as pd

In [36]:
data = pd.read_csv("../DataSet/fasttext_train_set.csv", sep="\t").iloc[:4000]
train = data.iloc[:3000]
train['label_ft'] = '__label__' + train['label'].astype(str)
train[['text', 'label_ft']].to_csv('train.csv', index=None, header=None, sep='\t')
test = data.iloc[3000:]

pd.read_csv('train.csv', header=None, sep='\t')

/Users/wangjie/Anaconda/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,0,1
0,2967 6758 339 2021 1854 3731 4109 3792 4149 15...,__label__2
1,4464 486 6352 5619 2465 4802 1452 3137 5778 54...,__label__11
2,7346 4068 5074 3747 5681 6093 1777 2226 7354 6...,__label__3
3,7159 948 4866 2109 5520 2490 211 3956 5520 549...,__label__2
4,3646 3055 3055 2490 4659 6065 3370 5814 2465 5...,__label__3
...,...,...
2995,600 2448 5510 4333 431 5858 6574 2602 6822 442...,__label__2
2996,4302 2799 5724 6227 4302 4145 2465 6962 5393 1...,__label__0
2997,7154 4893 3686 5192 1215 7292 2602 2685 3659 2...,__label__8
2998,2799 3370 3370 6242 2226 1919 2465 88 1999 289...,__label__0


In [42]:
# 文本分类
model = fasttext.train_supervised(input='train.csv', label_prefix='__label__',
                                         epoch=10, dim=32, lr=0.1, loss='softmax', word_ngrams=2, 
                                          min_count=2, bucket=1000000)
test_data = test['text'].values.tolist()
model.predict(test_data)[0][:3]

[['__label__0'], ['__label__1'], ['__label__1']]

In [44]:
# 训练词向量
embed_model = fasttext.train_unsupervised(input='train.csv', model='cbow', min_count=2, dim=32, ws=3)

In [50]:
embed_model['6640']

array([ 0.33897159, -0.7122523 ,  0.07081984, -0.16943555,  0.9051555 ,
       -0.20676735, -0.31139496, -0.34154943, -0.9285196 ,  0.13740878,
       -0.83647037,  0.16830571,  0.04207527, -0.43230844,  0.13847707,
       -0.16509397,  0.0680689 , -0.01457736,  0.44141197,  0.47386608,
        0.43244776,  1.1654117 , -0.72536886,  0.69115084, -0.7064338 ,
        0.2890822 , -0.1784637 , -1.0929071 ,  1.5237536 ,  0.7414258 ,
        0.29959834,  1.4486113 ], dtype=float32)